In [18]:
# Install required libraries
!pip install langchain transformers torch --quiet

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load lightweight model
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150, temperature=0.7)

llm = HuggingFacePipeline(pipeline=text_gen)

# ✅ Dummy knowledge base
knowledge_base = {
    "hook": "A hook is the opening sentence of an essay meant to grab the reader's attention.",
    "thesis": "A thesis statement clearly presents the main idea of your essay in one or two sentences.",
    "conclusion": "A strong conclusion summarizes your argument and provides closure to your essay.",
    "transition": "Transition words help to smoothly connect ideas between sentences and paragraphs.",
    "passive voice": "Passive voice makes sentences less direct. Use active voice for clarity."
}

# ✅ Tool 1: Writing Knowledge Tool
def writing_knowledge_tool(query: str) -> str:
    for key, value in knowledge_base.items():
        if key in query.lower():
            return f"{key.title()}: {value}"
    return "No relevant writing tip found in the knowledge base."

# ✅ Tool 2: Grammar Check Tool
def grammar_check_tool(text: str) -> str:
    suggestions = []
    if "ain't" in text:
        suggestions.append("Consider replacing 'ain't' with 'is not' or 'are not'.")
    if "I has" in text:
        suggestions.append("Use 'I have' instead of 'I has'.")
    return "\n".join(suggestions) if suggestions else "No major grammar issues detected."

# ✅ Example user input
user_question = "What is a hook in writing and check the sentence 'I has a dream that ain't bad.'"

# Run tools manually
knowledge_response = writing_knowledge_tool(user_question)
grammar_feedback = grammar_check_tool(user_question)

# Prompt Template
template = PromptTemplate(
    input_variables=["question", "knowledge", "grammar"],
    template="""
You are an intelligent AI Writing Assistant.

User Question: {question}

Writing Knowledge Base Info:
{knowledge}

Grammar Feedback:
{grammar}

Based on the above, generate a helpful and concise response to assist the user.
"""
)

final_prompt = template.format(
    question=user_question,
    knowledge=knowledge_response,
    grammar=grammar_feedback
)

# ✅ Generate Response
response = llm.invoke(final_prompt)
print("🤖 Final Response:\n", response)


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🤖 Final Response:
 
You are an intelligent AI Writing Assistant.

User Question: What is a hook in writing and check the sentence 'I has a dream that ain't bad.'

Writing Knowledge Base Info:
Hook: A hook is the opening sentence of an essay meant to grab the reader's attention.

Grammar Feedback:
Consider replacing 'ain't' with 'is not' or 'are not'.
Use 'I have' instead of 'I has'.

Based on the above, generate a helpful and concise response to assist the user.

Note: As part of your writing development process, make sure you are using correct grammar.

The hook is a very useful tool for writing a good story. It allows you to create a concise and concise sentence that conveys a clear message about your subject matter.

I have a dream that ain't bad. I have a dream that ain't bad and I don't want to die. I have a dream that is not bad. I have a dream that ain't bad and I don't want to die. I have a dream that is not bad and I don't want to die. I have a dream that is not bad and I don'